### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Dependency

In [ ]:
!pip install transformers[sentencepiece] git+https://github.com/huggingface/peft.git accelerate bitsandbytes loralib fire

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-rf17azu8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-rf17azu8
  Resolved https://github.com/huggingface/peft.git to commit b5b902368d2a7f349783a96d9d3c686ea0ca3a05
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


###Class Stream

In [ ]:
import gc
import traceback
from queue import Queue
from threading import Thread

import torch
import transformers

class Stream(transformers.StoppingCriteria):
    def __init__(self, callback_func=None):
        self.callback_func = callback_func

    def __call__(self, input_ids, scores) -> bool:
        if self.callback_func is not None:
            self.callback_func(input_ids[0])
        return False

class Iteratorize:
    """
    Transforms a function that takes a callback
    into a lazy iterator (generator).
    """

    def __init__(self, func, kwargs={}, callback=None):
        self.mfunc = func
        self.c_callback = callback
        self.q = Queue()
        self.sentinel = object()
        self.kwargs = kwargs
        self.stop_now = False

        def _callback(val):
            if self.stop_now:
                raise ValueError
            self.q.put(val)

        def gentask():
            try:
                ret = self.mfunc(callback=_callback, **self.kwargs)
            except ValueError:
                pass
            except:
                traceback.print_exc()
                pass

            self.q.put(self.sentinel)
            if self.c_callback:
                self.c_callback(ret)

        self.thread = Thread(target=gentask)
        self.thread.start()

    def __iter__(self):
        return self

    def __next__(self):
        obj = self.q.get(True, None)
        if obj is self.sentinel:
            raise StopIteration
        else:
            return obj

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop_now = True

###Class Promter

In [ ]:
import json
import os.path as osp
from typing import Union

class Prompter:
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        template_name = "alpaca"
        self.template = {
            "description": "Template used by Alpaca-LoRA.",
            "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
            "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
            "response_split": "### Response:"
        }
        if self._verbose:
            print(f"Using prompt template {template_name}: {self.template['description']}")

    def generate_prompt(self, instruction: str, input: Union[None, str] = None) -> str:
        if input:
            res = self.template["prompt_input"].format(instruction=instruction, input=input)
        else:
            res = self.template["prompt_no_input"].format(instruction=instruction)
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:  # noqa: E722
    pass

### Load Base Model and Lora

In [ ]:
import os
import sys

import torch
import transformers
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel, LoraConfig

base_model = '/content/drive/MyDrive/BaseModel/openthaigpt-1.0.0-7b-chat'
lora_weight = '/content/drive/MyDrive/TrainModel/Com_Laws'

adapter_config = LoraConfig(
    peft_type="LORA",
    base_model_name_or_path=base_model,
    inference_mode=True,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["v_proj", "q_proj"]
)

lora_8bit = True

tokenizer = LlamaTokenizer.from_pretrained(base_model)

device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=lora_8bit,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    model = PeftModel.from_pretrained(
        model,
        lora_weight,
        config=adapter_config,
        torch_dtype=torch.float16
    )

elif device == "cpu":
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        device_map={"": device},
        torch_dtype=torch.float16
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weight,
        config=adapter_config,
        torch_dtype=torch.float16
    )

model.config.pad_token_id = tokenizer.pad_token_id = 2
model.config.bos_token_id = 1
model.config.eos_token_id = 2

if not lora_8bit:
    model.half()

model.eval()

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

###Evaluate by calling a function

In [ ]:
def text_evaluate(
    instruction,
    input=None,
    num_beams=4,
    repetition_penalty=2,
    no_repeat_ngram_size=5,
    max_new_tokens=128,
    stream_output=True,
    **kwargs,
):
    prompter = Prompter(verbose=True)
    prompt = prompter.generate_prompt(instruction, input)
    inputs = tokenizer(prompt, padding=True, return_tensors="pt")

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = 0

    generation_config = GenerationConfig(
        num_beams=num_beams,
        pad_token_id=tokenizer.pad_token_id,
        **kwargs,
    )

    generate_params = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "generation_config": generation_config,
        "return_dict_in_generate": True,
        "output_scores": True,
        "max_new_tokens": max_new_tokens,
    }

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    return tokenizer.decode(s, skip_special_tokens=True).split("### Response:")[1].strip()

###Prompt input Test


In [ ]:
question = 'นาย A ทำงานให้กับธนาคารและล่วงรู้มาตรการรักษาความปลอดภัยในการเข้าถึงระบบคอมพิวเตอร์ของธนาคาร เขานำข้อมูลวิธีการเข้าถึงระบบนี้ไปเผยแพร่ในเว็บไซต์ออนไลน์ ทำให้แฮกเกอร์สามารถนำข้อมูลนี้ไปใช้ก่ออาชญากรรมผิดกฎหมายไหม และจะรับโทษอะไร'

print("Question : ", question)
print("Answer : ", text_evaluate(question))

Question :  นาย A ทำงานให้กับธนาคารและล่วงรู้มาตรการรักษาความปลอดภัยในการเข้าถึงระบบคอมพิวเตอร์ของธนาคาร เขานำข้อมูลวิธีการเข้าถึงระบบนี้ไปเผยแพร่ในเว็บไซต์ออนไลน์ ทำให้แฮกเกอร์สามารถนำข้อมูลนี้ไปใช้ก่ออาชญากรรมผิดกฎหมายไหม และจะรับโทษอะไร
Using prompt template alpaca: Template used by Alpaca-LoRA.
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
นาย A ทำงานให้กับธนาคารและล่วงรู้มาตรการรักษาความปลอดภัยในการเข้าถึงระบบคอมพิวเตอร์ของธนาคาร เขานำข้อมูลวิธีการเข้าถึงระบบนี้ไปเผยแพร่ในเว็บไซต์ออนไลน์ ทำให้แฮกเกอร์สามารถนำข้อมูลนี้ไปใช้ก่ออาชญากรรมผิดกฎหมายไหม และจะรับโทษอะไร

### Response:

Answer :  นาย A อาจต้องรับโทษจำคุก 5 ปี และปรับเงิน 100,000 บาท หากเขาถูกตัดสินว่ามีความผิดฐานเผยแพร่มาตรการรักษาความปลอดภัยในการเข้าถึงระบบคอมพิวเตอร์ของธนาคารโดยผิดกฎหมาย เนื่องจากการกระทำดังกล่าวเป็นอันตรายต่อธนาคารและผู้ใช้บริการของธนาคาร นอกจากนี้ นาย A อาจต้องรับโทษจำคุก 5 ปี และปรับเงิน 100,000 บาท หากเขาถูกตัดสินว่ามีความผ